In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-02.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/working_parquet_format.pdf
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/data_dictionary_trip_records_hvfhs.pdf
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-10.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-05.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/taxi_zone_lookup.csv
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-06.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-04.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-09.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-12.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv_tripdata_2021-08.parquet
/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021/fhvhv

# 1. Loading all files in one dataframe

In [2]:
# Installing PySpark
!pip install pyspark

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=2d5a4673f7fde02e393860c90305b7beda8bd7ab0b70868183cf3af64b081523
  Stored in directory: /root/.cache/pip/wheels/07/fb/67/b9f2c0242d156eaa136b45ae4fd99d3e7c0ecc2acfd26f47b9
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


## 1.1 Creating SparkSession

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, floor

In [4]:
spark = SparkSession.builder.master("local[*]").appName("trips_data").getOrCreate()

bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/26 19:06:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:
default_path = "/kaggle/input/uber-nyc-forhire-vehicles-trip-data-2021"

In [7]:
df = spark.read.parquet(f"{default_path}/fhvhv_tripdata_2021-*.parquet")

In [8]:
df.is_cached

False

In [9]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [10]:
df.rdd.getNumPartitions()

36

In [11]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [12]:
df.count()

174596652

# 2. Separating files by type of license

As described in the dataset documentation:

**Field name:** hvfhs_license_num

**Description:** the TLC license number of the HVFHS base or business as of September 2019, the HVFHS licensees are the following: HV0002: Juno, HV0003: Uber, HV0004: Via and HV0005: Lyft

In [13]:
# Checking how many different groups the dataset have
df.groupBy("hvfhs_license_num").count().show()

+-----------------+---------+
|hvfhs_license_num|    count|
+-----------------+---------+
|           HV0004|   891819|
|           HV0005| 47575769|
|           HV0003|126129064|
+-----------------+---------+



One can notice that we have no licese number that ends with 02, then we're going to work with the existing license numbers.

In [14]:
df_uber = df.filter(col("hvfhs_license_num") == 'HV0003')

In [15]:
df_via = df.filter(col("hvfhs_license_num") == 'HV0004')

In [16]:
df_lyft = df.filter(col("hvfhs_license_num") == 'HV0005')

# 3. Setting aside only rides that were not shared using UDF

In [17]:
df.groupBy("shared_match_flag").count().show()

+-----------------+---------+
|shared_match_flag|    count|
+-----------------+---------+
|                Y|   107080|
|                N|174489572|
+-----------------+---------+



In [18]:
# Create Temporary table in PySpark
df.createOrReplaceTempView("df_view")

In [19]:
# Creating the UDF
only_not_shared = """

    SELECT * 
    FROM df_view
    WHERE shared_match_flag != "Y"

"""

In [20]:
df_not_shared = spark.sql(only_not_shared)

In [21]:
df_not_shared.groupBy("shared_match_flag").count().show()

+-----------------+---------+
|shared_match_flag|    count|
+-----------------+---------+
|                N|174489572|
+-----------------+---------+



In [22]:
df_not_shared.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

# 4. Selecting trips which the driver received more than 10 dollars

In [23]:
# It rounds the numbers to the nearest multiple of 10. If it's less than 10, it'll be 0
df = df.withColumn("grouped_driver_pay", floor(df["driver_pay"]/10)*10)


In [27]:
# It groups the rounded values and counts how many there are in each group
df.groupBy("grouped_driver_pay").count().show()

+------------------+--------+
|grouped_driver_pay|   count|
+------------------+--------+
|               270|    1400|
|               730|       7|
|                 0|55453505|
|               330|     654|
|               130|   35572|
|               830|       1|
|                50| 2363075|
|               600|      17|
|               650|      15|
|               110|   79833|
|               190|    6835|
|               420|     159|
|               450|     103|
|               760|       5|
|               370|     258|
|               320|     624|
|               150|   17282|
|               170|   10086|
|               430|     134|
|              1040|       3|
+------------------+--------+
only showing top 20 rows



In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [31]:
df.filter(df.grouped_driver_pay >10).count()

52464902

In [33]:
df_over_10 = df.filter(df.grouped_driver_pay >10)

In [34]:
df_over_10.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+------------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|grouped_driver_pay|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+-------